First R-based experiments showed the datasets are very very large, and R isn't suited for the pre-processing steps.
I'm going to try to use Python instead to generate my initial dataset



In [1]:
from collections import defaultdict
import string
import sys
import re
import copy
from stemming.porter2 import stem

In [2]:
def find_ngrams(input_list, n):
  return zip(*[input_list[i:] for i in range(n)])

In [3]:
re_tweetfilter = re.compile('[^a-z0-9#@]')
def lineProcess(tdm, inline, docNumber, ngrams = 1):
    # lowercase, remove punctuation, numbers
    inline = re_tweetfilter.sub(' ',inline.lower()).split()
    # stem anything that isn't name or hashtag
    for i in range(len(inline)):
        inline[i]=stem(inline[i]) if not inline[i].startswith(('@','#')) else 'x'+inline[i]
    # find the n-grams
    ngramlist=[]
    for j in range(ngrams):
        ngramlist+=find_ngrams(inline,j+1)
    # and finally add to the dictionary
    for k in ngramlist:
        k=':'.join(k)
        if k not in tdm: tdm[k]={}
        if docNumber not in tdm[k]: tdm[k][docNumber] = 0
        tdm[k][docNumber]+=1
    return tdm

In [5]:
def purgeSparseTdm(tdm, minterms=1):
    tmpTdm = dict(tdm)
    for ngram in tdm:
        cnt=sum([tdm[ngram][x] for x in tdm[ngram]])
        if cnt<minterms:
            del tmpTdm[ngram]
    return copy.copy(tmpTdm)

In [6]:
def dtmToTdm(inDtm):
    outTdm={}
    for lineNumber in inDtm:
        for word in inDtm[lineNumber]:
            if word not in outTdm: outTdm[word]={}
            outTdm[word][lineNumber]=inDtm[lineNumber][word]
    return outTdm
    

In [7]:
source_file = '/Users/ratb3rt/Downloads/twitter_data/en_US/en_US.twitter.txt'
gDocNumber = 0
myTdm={}
with open(source_file) as f:
    for content in f:
        gDocNumber+=1
        myTdm=lineProcess(myTdm, content, gDocNumber, 3)




KeyboardInterrupt: 

In [1]:
myTdm2=purgeSparseTdm(myTdm, 500)
myTdm = None

NameError: name 'purgeSparseTdm' is not defined

In [121]:
# write my tdm to disk
import json
with open('en_US.twitter.processed.json', 'w') as f:
    for x in myTdm2:
        f.write(json.dumps({x:myTdm2[x]}, indent = 2, ensure_ascii=True))